### What is this?
In #7 we tried to use Seq2Seql but we couldn't figure it out.
Here we're going to use [working example](https://github.com/nicolas-ivanov/debug_seq2seq) step by step, so that we should be able to know what's going on.

### Data
We gonna use very small data set for faster iterations. Once we're done with whole process, we will increase data size.

### word2vec
Here we convert our vocabrary to vector, so that we can have smaller dimension and similarities between words.

In [1]:
## Read data file
import MeCab
EOS_SYMBOL = "$$$"
m = MeCab.Tagger()
m.parse('')

f = open("/Users/higepon/Dropbox/Machine-Learning/dont_remove_data/webdb.txt")
line = f.readline() 
lines = []
while line:
    res = m.parseToNode(line)

    words = []
    while res:
        words.append(res.surface)
        res = res.next
    words.append(EOS_SYMBOL)
    lines.append(words)
    line = f.readline()
print(lines[299])


['', '\u3000', '週', 'に', '一', '度', 'の', '週', '次', 'レビュー', 'で', '、', '上記', 'の', 'リスト', 'の', 'アイテム', 'を', 'おおよそ', '次', 'の', 'よう', 'に', '分類', 'する', '。', '', '$$$']


In [3]:
from gensim.models import Word2Vec
from itertools import tee

window = 5              # window is the maximum distance between the current and predicted word within a sentence.
min_count = 1           # ignore all words with total frequency lower than this.
max_vocab_size = 20000  # limit RAM during vocabulary building; if there are more unique words than this, then prune the infrequent ones. Every 10 million word types need about 1GB of RAM. Set to None for no limit (default).
vec_size = 256          # size is the dimensionality of the feature vectors.
workers = 5             # use this many worker threads to train the model (=faster training with multicore machines

wv_model = Word2Vec(window=window,
                 min_count=min_count,
                 max_vocab_size=max_vocab_size,
                 size=vec_size,
                 workers=workers)
print("Word2Vec model", wv_model)

# Using Japanese data file instead
# Input for build_vocab is generator for [['excuse', 'me', '?', '$$$']], where $$$ is EOS and ### is Empty token
#lines = [
#    ["I", "am", "John", ".", "$$$"],
#    ["I", "don't", "want", "to", "spoil", "the", "party", ".", "EOS"]
#]
tokenized_lines, tokenized_lines_for_train, tokenized_lines_for_voc = tee(lines, 3)
wv_model.build_vocab(tokenized_lines)
wv_model.train(tokenized_lines_for_train)

print(wv_model.most_similar(positive=['エンジニア'], negative=[], topn=3))


Word2Vec model Word2Vec(vocab=0, size=256, alpha=0.025)
[('。', 0.978071928024292), ('て', 0.9771312475204468), ('マネージャ', 0.977081298828125)]


In [4]:
from collections import Counter

EMPTY_TOKEN = "###"
token_counter = Counter()
for line in tokenized_lines_for_voc:
    for token in line:
        token_counter.update([token])

token_voc = [token for token, _ in token_counter.most_common()[:max_vocab_size]]
token_voc.append(EMPTY_TOKEN)
index_to_token = dict(enumerate(token_voc))
print(index_to_token)



{0: '', 1: '。', 2: 'の', 3: '$$$', 4: 'を', 5: 'に', 6: 'て', 7: 'が', 8: 'は', 9: '、', 10: 'で', 11: 'と', 12: 'し', 13: 'こと', 14: 'ない', 15: 'た', 16: 'する', 17: 'う', 18: 'も', 19: 'ある', 20: 'だ', 21: '\u3000', 22: 'いる', 23: 'か', 24: 'な', 25: '」', 26: '「', 27: '1', 28: 'ば', 29: 'だろ', 30: '-', 31: '的', 32: 'から', 33: 'よう', 34: '勉強', 35: 'さ', 36: 'ユーザー', 37: 'い', 38: 'なる', 39: 'プロダクト', 40: '#', 41: 'その', 42: 'など', 43: '時間', 44: '自分', 45: 'という', 46: '人', 47: 'チーム', 48: '筆者', 49: '/', 50: '何', 51: 'れ', 52: 'もの', 53: 'よい', 54: 'や', 55: '）', 56: 'マネージャ', 57: 'エンジニア', 58: '問題', 59: 'たい', 60: 'しよ', 61: 'ので', 62: '（', 63: '良い', 64: 'この', 65: 'これ', 66: '常に', 67: 'たり', 68: 'できる', 69: 'つ', 70: '必要', 71: '思う', 72: 'れる', 73: 'かも', 74: '場合', 75: 'しれ', 76: '>', 77: ':', 78: 'テスト', 79: 'なく', 80: '機能', 81: '性', 82: '次', 83: 'それ', 84: 'なっ', 85: 'しまう', 86: '.', 87: 'プロダクティビティ', 88: 'たら', 89: 'ため', 90: 'とき', 91: 'A', 92: 'そこ', 93: '2', 94: '？', 95: 'として', 96: '可能', 97: '技術', 98: 'カバン', 99: 'まで', 100: '<', 101: 'B', 102

### The model

In [ ]:
from seq2seq.models import SimpleSeq2Seq
from keras.models import Sequential

print("token_size", len(index_to_token))

#seq2seq = Sequential()
model = SimpleSeq2Seq(
    input_dim=vec_size,
    input_length=16,
    hidden_dim=512,
    output_dim=len(index_to_token),
    output_length=6,
    depth=1
)

#model.add(seq2seq)
model.compile(loss='categorical_crossentropy', optimizer='rmsprop')

print("model compile done")

token_size 2551
